In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/newsdata/train_data.csv")
test = pd.read_csv("/kaggle/input/newsdata/test_data.csv")
topic = pd.read_csv("/kaggle/input/newsdata/topic_dict.csv")
display(train,test,topic)

In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
text = list(all_data["title"])
text

In [ ]:
#pip install konlpy

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()
kkma_list = []
for i in text:
    k = kkma.morphs(i)
    kkma_list.append(k)
kkma_list

In [ ]:
kkma_list = np.array(kkma_list)
kkma_list

In [ ]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(kkma_list)

In [ ]:
text = tk.texts_to_sequences(kkma_list) 
text

In [ ]:
from keras.preprocessing.sequence import pad_sequences
pad_text = pad_sequences(text, padding = "post")
pad_text

In [ ]:
pad_text.shape

In [ ]:
train_text = pad_text[:len(train)]
test_text = pad_text[len(train):]

In [ ]:
len(tk.word_index)

In [ ]:
result_text = 0

from sklearn.model_selection import StratifiedKFold
from keras import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import Adam

skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 82)  
for train_index, valid_index in skf.split(train_text,train["topic_idx"]):
    es = EarlyStopping(patience = 15, verbose = 1) 
    mc = ModelCheckpoint("best.h5",save_best_only = True, verbose = 1)
   
    X_train, X_valid = train_text[train_index], train_text[valid_index] 
    y_train, y_valid = train["topic_idx"].iloc[train_index], train["topic_idx"].iloc[valid_index] 
    
    model = Sequential()
    model.add(Embedding(27110,15, input_length = 26))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(7,activation = "softmax"))
   
    model.compile(metrics=["acc"], optimizer = Adam(lr = 0.0005), loss = "sparse_categorical_crossentropy")
    history = model.fit(X_train, y_train, validation_data = (X_valid,y_valid), epochs = 500, callbacks = [es,mc])
    model.load_weights("best.h5")
    result_text += model.predict(test_text, verbose = 1) / 5
    

In [ ]:
sub = pd.read_csv("/kaggle/input/newsdata/sample_submission.csv")
sub["topic_idx"] = result_text.argmax(1)
sub

In [ ]:
sub.to_csv("sub_konlpy4.csv",index=False)